## Import

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
from selenium import webdriver
from bs4 import BeautifulSoup
from utils import *

import requests
import streamlit as st
import pandas as pd
import numpy as np
import OpenDartReader
import warnings
import time, datetime
import re, os

warnings.filterwarnings('ignore')

* API setting

In [ ]:
start_dt = '2023-05-01'
end_dt = '2023-06-01'

In [ ]:
head_df = main(start_dt, end_dt, opt = 'IB전략')
form_1, form_2, form_3 = main(start_dt, end_dt, opt = '기업금융1부')

* 디버깅

In [3]:
from selenium.webdriver.chrome.options import Options

In [4]:
start_dt = '2022-12-16'
end_dt = '2023-01-16'

In [5]:
api_key = '1b39652cef07f626c9d37375edf582ee51b1407f'
dart = OpenDartReader(api_key)

# C=발행공시, D=지분공시
market_dict = {"Y": "코스피","K": "코스닥", "N": "코넥스", "E": "기타"}

info_df = dart.list(start=start_dt, end=end_dt, kind_detail='C001')
info_df = pd.concat([info_df, dart.list(start=start_dt, end=end_dt, kind_detail='G002')])

info_df = info_df.loc[[True if "증권발행실적보고서" in x else False for x in info_df.report_nm]]
info_df = info_df.loc[info_df.corp_cls.isin(['Y', 'K'])]
info_df.corp_cls = info_df.corp_cls.map(market_dict)

In [6]:
options = Options()
options.add_argument('--headless')

In [7]:
#driver = webdriver.Chrome(options = options)
driver = webdriver.Chrome(options = options)
driver.set_window_size(1980, 1080)

In [8]:
api_key = '1b39652cef07f626c9d37375edf582ee51b1407f'
dart = OpenDartReader(api_key)

# C=발행공시, D=지분공시
market_dict = {"Y": "코스피","K": "코스닥", "N": "코넥스", "E": "기타"}

info_df = dart.list(start=start_dt, end=end_dt, kind_detail='C001')
info_df = pd.concat([info_df, dart.list(start=start_dt, end=end_dt, kind_detail='G002')])

info_df = info_df.loc[[True if "증권발행실적보고서" in x else False for x in info_df.report_nm]]
info_df = info_df.loc[info_df.corp_cls.isin(['Y', 'K'])]
info_df.corp_cls = info_df.corp_cls.map(market_dict)

# kind
first_df = kind_main(driver, info_df, start_dt, end_dt)

In [9]:
# ipo stock
driver = webdriver.Chrome(options = options)
driver.set_window_size(1920, 1080)

ipo_df = ipo_main(driver, info_df)
first_df = pd.merge(first_df, ipo_df, on = 'corp_name')

In [10]:
# 38커뮤니케이션
outer_df = get_38(start_dt, end_dt)
second_df = pd.merge(first_df, outer_df, on = 'stock_code', how = 'inner')

# DART
third_df = get_dd(dart, second_df)
third_df, fourth_df = get_d_tables(dart, third_df)

In [12]:
opt = "IB전략"
head_df = change_form(third_df, opt)

opt = "ㅈㄷㄱ"
form_1 = change_form(fourth_df, opt, 1)
form_2 = change_form(third_df, opt, 2)
form_3 = change_form(fourth_df, opt, 3)

In [20]:
head_df.sort_values("수요예측(시작일)").to_excel("IB전략컨설팅부-2305.xlsx", index = False, encoding = 'CP949')
form_1.sort_values("상장일").to_excel("기업금융1부-01-리그테이블-2305.xlsx", index = False, encoding = 'CP949')
form_2.sort_values("청약일").to_excel("기업금융1부-02-통합집계-2305.xlsx", index = False, encoding = 'CP949')
form_3.to_excel("기업금융1부-03-summary-2305.xlsx", index = False, encoding = 'CP949')

In [ ]:
cd C:\Users\Administrator\Documents\GitHub\opendart\ipynb 

In [ ]:
streamlit run Corporate-Finance-1dept.py

In [ ]:
streamlit run IB-strategy.py